In [10]:
from llama_index.legacy.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)
import re
import logging
from llama_index.legacy.llms import OpenAI
import os
from tqdm import trange
import json
from llama_index.legacy.schema import TextNode
from llama_index.legacy.node_parser import SentenceWindowNodeParser, SemanticSplitterNodeParser
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from groq import Groq

## load env variables
GROQ_API_KEY       = os.environ["GROQ_API_KEY"]
CHAT_MODEL         = "llama3-70b-8192"
client = Groq()

In [4]:
with open("../data/combined_txts/combined_output.txt", "r", encoding='utf-8') as fin:
    contexts = fin.read()
sections = [section for section in contexts.split("\n\n\n") if section]

['Based on national health statistics from the Ministry of Health in 2019, Singapore, ischemic heart disease (lack of blood circulation to heart muscles) is the third most common cause of death, contributing to 18.8% of total mortality. \nA person with heart disease is prone to a heart attack, which could result in cardiac arrest and sudden death. According to a study conducted in Singapore*, the crude incidence rate of Out-of-Hospital Cardiac Arrest (OHCA) in 2019 was 56.7 per 100,000 population in 2019 (3233 casualties), of which, 2239 cases (74%) of cardiac arrests occur in the home. Bystander CPR was performed on 1937 casualties (60%) and bystander defibrillation was performed on 340 (10.5%) of them. Out of 200 survivors (6.2%), 156 (4.8%) of them survived to be discharged with good-to-moderate neurological functions. \nSurvival from sudden death can be maximized with the prompt application of basic life-saving skills of cardio-pulmonary resuscitation (CPR) and use of automated ext

In [8]:
with open("../data/final_draft_2_pairs.json", "r", encoding="utf-8") as fin:
    pairs = json.load(fin)
questions = [pair['Question'] for pair in pairs]
questions


['What is the percentage of total mortality in Singapore attributed to ischemic heart disease, according to national health statistics from the Ministry of Health in 2019?',
 'What is the most common location where cardiac arrests occur, based on a study conducted in Singapore in 2019?',
 'What is the percentage of Out-of-Hospital Cardiac Arrest (OHCA) cases in which bystander CPR was performed, according to the study?',
 'What is the percentage of survivors who survived to be discharged with good-to-moderate neurological functions, out of the total number of OHCA cases?',
 'What are the two basic life-saving skills that can be performed by anyone, anywhere, and anytime to maximize survival from sudden death, according to the text?',
 'What is the primary function of the right side of the heart in the circulatory process?',
 'Which blood vessels carry oxygen-rich blood from the lungs to the left side of the heart?',
 'What are the vital organs that receive oxygen-rich blood from the le

In [ ]:
DEFAULT_QUERY_DOC_RELEVANCE_PROMPT = \
'''You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Query: How to plant a tree?
Document: """Cars were invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.[3][4][5] Cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages.[6] In Europe and other parts of the world, demand for automobiles did not increase until after World War II.[7] The car is considered an essential part of the developed economy."""
Relevant: No

Query: Has the coronavirus vaccine been approved?
Document: """The Pfizer-BioNTech COVID-19 vaccine was approved for emergency use in the United States on December 11, 2020."""
Relevant: Yes

Query: What is the capital of France?
Document: """Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Beyond such landmarks as the Eiffel Tower and the 12th-century, Gothic Notre-Dame cathedral, the city is known for its cafe culture and designer boutiques along the Rue du Faubourg Saint-Honoré."""
Relevant: Yes

Query: What are some papers to learn about PPO reinforcement learning?
Document: """Proximal Policy Optimization and its Dynamic Version for Sequence Generation: In sequence generation task, many works use policy gradient for model optimization to tackle the intractable backpropagation issue when maximizing the non-differentiable evaluation metrics or fooling the discriminator in adversarial learning. In this paper, we replace policy gradient with proximal policy optimization (PPO), which is a proved more efficient reinforcement learning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We demonstrate the efficacy of PPO and PPO-dynamic on conditional sequence generation tasks including synthetic experiment and chit-chat chatbot. The results show that PPO and PPO-dynamic can beat policy gradient by stability and performance."""
Relevant: Yes

Query: Explain sentence embeddings
Document: """Inside the bubble: exploring the environments of reionisation-era Lyman-α emitting galaxies with JADES and FRESCO: We present a study of the environments of 16 Lyman-α emitting galaxies (LAEs) in the reionisation era (5.8<z<8) identified by JWST/NIRSpec as part of the JWST Advanced Deep Extragalactic Survey (JADES). Unless situated in sufficiently (re)ionised regions, Lyman-α emission from these galaxies would be strongly absorbed by neutral gas in the intergalactic medium (IGM). We conservatively estimate sizes of the ionised regions required to reconcile the relatively low Lyman-α velocity offsets (ΔvLyα<300kms−1) with moderately high Lyman-α escape fractions (fesc,Lyα>5%) observed in our sample of LAEs, indicating the presence of ionised ``bubbles'' with physical sizes of the order of 0.1pMpc≲Rion≲1pMpc in a patchy reionisation scenario where the bubbles are embedded in a fully neutral IGM. Around half of the LAEs in our sample are found to coincide with large-scale galaxy overdensities seen in FRESCO at z∼5.8-5.9 and z∼7.3, suggesting Lyman-α transmission is strongly enhanced in such overdense regions, and underlining the importance of LAEs as tracers of the first large-scale ionised bubbles. Considering only spectroscopically confirmed galaxies, we find our sample of UV-faint LAEs (MUV≳−20mag) and their direct neighbours are generally not able to produce the required ionised regions based on the Lyman-α transmission properties, suggesting lower-luminosity sources likely play an important role in carving out these bubbles. These observations demonstrate the combined power of JWST multi-object and slitless spectroscopy in acquiring a unique view of the early stages of Cosmic Reionisation via the most distant LAEs."""
Relevant: No

Query: {query}
Document: """{document}"""
Relevant:
'''

def extract_answer(input_string):
    """
    Extracts and returns the JSON data from a given input string.

    This function attempts to extract a JSON object from the provided input string.
    The input string is expected to contain JSON data starting with `{` and ending
    with `}`. If a valid JSON object is not found, it attempts to locate a JSON object
    containing a 'questions' key using regular expressions.

    Args:
        input_string (str): The input string that potentially contains JSON data.

    Returns:
        dict: A dictionary representing the extracted JSON data.

    Raises:
        ValueError: If no JSON data is found in the input string.
        json.JSONDecodeError: If the JSON data is malformed and cannot be parsed.

    """
    # Find the start and end indices of the JSON data within the input string
    # Assuming the JSON data starts with '{' and ends with '}'
    json_start = input_string.find('{')
    json_end = input_string.rfind('}') + 1
    
    # If either the start or end index is not found, raise an error
    if json_start == -1 or json_end == -1:
        raise ValueError("Invalid input: No JSON data found.")

    # Extract the substring that potentially contains the JSON data
    json_data = input_string[json_start:json_end]
    
    try:
        # Attempt to convert the JSON string to a Python dictionary
        data_dict = json.loads(json_data)
        return data_dict
    
    except json.JSONDecodeError:
        # If JSON decoding fails, search for a JSON object containing the 'questions' key
        # Using regex to match a pattern that includes the 'questions' key
        pattern = r'{\s*"questions":\s*\[.*?\]\s*}'
        match = re.search(pattern, input_string, re.DOTALL)

        if match:
            # If a match is found, extract the matched JSON string and convert it to a dictionary
            data_json_str = match.group(0)
            data_dict = json.loads(data_json_str)
            return data_dict

        # If no valid JSON is found, the function will Log an error
        else:
            logging.error("No dictionary with 'questions' as a key found in this input string. Error by LLM")
            return {"error": "No dictionary with questions found"}


def generate_bool(context, question, relevance_prompt, client):
    # Define a Pydantic model for the expected answer structure
    class AnswerList(BaseModel):
        answers: list = Field(description="Answer to the question with reference to the content in the text corpus")
    
    # Initialize a JSON output parser using the defined Pydantic model
    parser = JsonOutputParser(pydantic_object=AnswerList)

    # Prepare the prompt using the provided answer prompt template, text, and list of questions
    prompt = PromptTemplate(
        template=relevance_prompt,
        input_variables=["text", "question_list"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    ) 
    
    # Format the final prompt with the actual text data and question list
    final_prompt = prompt.format(query=question, document=context)

    # Generate the completion by interacting with the language model API
    completion = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {
                "role": "user",
                "content": final_prompt
            }
        ],
        temperature=0,  # Control the randomness of the output (lower means less random)
        max_tokens=1024,  # Limit the response length
        top_p=1,  # Nucleus sampling parameter (1 means only the most likely tokens are considered)
        stream=True,  # Enable streaming of the response chunks
        stop=None,  # Define stopping conditions (None means no stopping condition)
    )

    # Initialize an empty string to accumulate the response content
    answer = ''''''
    for chunk in completion:
        # Append each chunk of content to the answer string
        answer += chunk.choices[0].delta.content or ""
    
    # Extract the answers from the accumulated response content
    answer_dict = extract_answer(answer)

    # Log an error if the response contains an 'error' key
    if "error" in answer_dict:
        logging.error(f"{answer_dict['error']}")
    
    # Return the dictionary containing the generated answers
    return answer_dict



def generate_ce_fine_tuning_dataset(contexts, questions_list, qa_doc_relevance_prompt):

    for question in trange(questions_list):
        if question != "":
            for context in contexts:
                
                msg_prompt = qa_doc_relevance_prompt.format(
                    query=question, document=context
                )
                response = llm.complete(msg_prompt)
                result = response.text.strip().lower()


                if result == "yes":
                    question_row = CrossEncoderFinetuningDatasetSample(
                        query=question, context=node_content, score=1
                    )
                    ce_dataset_list.append(question_row)
                    
                    
                elif result == "no":
                    question_row = CrossEncoderFinetuningDatasetSample(
                        query=question, context=node_content, score=0
                    )
                    ce_dataset_list.append(question_row)
                    
                else:
                    pass

    return ce_dataset_list

In [14]:
DEFAULT_QUERY_DOC_RELEVANCE_PROMPT = '''You are an Assistant responsible for helping detect whether the retrieved document is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved document is relevant to the query.

Query: How to plant a tree?
Document: """Cars were invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.[3][4][5] Cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages.[6] In Europe and other parts of the world, demand for automobiles did not increase until after World War II.[7] The car is considered an essential part of the developed economy."""
Relevant: No

Query: Has the coronavirus vaccine been approved?
Document: """The Pfizer-BioNTech COVID-19 vaccine was approved for emergency use in the United States on December 11, 2020."""
Relevant: Yes

Query: What is the capital of France?
Document: """Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Beyond such landmarks as the Eiffel Tower and the 12th-century, Gothic Notre-Dame cathedral, the city is known for its cafe culture and designer boutiques along the Rue du Faubourg Saint-Honoré."""
Relevant: Yes

Query: What are some papers to learn about PPO reinforcement learning?
Document: """Proximal Policy Optimization and its Dynamic Version for Sequence Generation: In sequence generation task, many works use policy gradient for model optimization to tackle the intractable backpropagation issue when maximizing the non-differentiable evaluation metrics or fooling the discriminator in adversarial learning. In this paper, we replace policy gradient with proximal policy optimization (PPO), which is a proved more efficient reinforcement learning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We demonstrate the efficacy of PPO and PPO-dynamic on conditional sequence generation tasks including synthetic experiment and chit-chat chatbot. The results show that PPO and PPO-dynamic can beat policy gradient by stability and performance."""
Relevant: Yes

Query: Explain sentence embeddings
Document: """Inside the bubble: exploring the environments of reionisation-era Lyman-α emitting galaxies with JADES and FRESCO: We present a study of the environments of 16 Lyman-α emitting galaxies (LAEs) in the reionisation era (5.8<z<8) identified by JWST/NIRSpec as part of the JWST Advanced Deep Extragalactic Survey (JADES). Unless situated in sufficiently (re)ionised regions, Lyman-α emission from these galaxies would be strongly absorbed by neutral gas in the intergalactic medium (IGM). We conservatively estimate sizes of the ionised regions required to reconcile the relatively low Lyman-α velocity offsets (ΔvLyα<300kms−1) with moderately high Lyman-α escape fractions (fesc,Lyα>5%) observed in our sample of LAEs, indicating the presence of ionised ``bubbles'' with physical sizes of the order of 0.1pMpc≲Rion≲1pMpc in a patchy reionisation scenario where the bubbles are embedded in a fully neutral IGM. Around half of the LAEs in our sample are found to coincide with large-scale galaxy overdensities seen in FRESCO at z∼5.8-5.9 and z∼7.3, suggesting Lyman-α transmission is strongly enhanced in such overdense regions, and underlining the importance of LAEs as tracers of the first large-scale ionised bubbles. Considering only spectroscopically confirmed galaxies, we find our sample of UV-faint LAEs (MUV≳−20mag) and their direct neighbours are generally not able to produce the required ionised regions based on the Lyman-α transmission properties, suggesting lower-luminosity sources likely play an important role in carving out these bubbles. These observations demonstrate the combined power of JWST multi-object and slitless spectroscopy in acquiring a unique view of the early stages of Cosmic Reionisation via the most distant LAEs."""
Relevant: No

Query: {query}
Document: """{document}"""
Relevant:
'''


def generate_questions(query, doc, client):
    # Prepare the prompt using the provided question prompt template and input text
    prompt = PromptTemplate(
        template=DEFAULT_QUERY_DOC_RELEVANCE_PROMPT,
        input_variables=["query", "document"],
    ) 
    
    # Format the final prompt with the actual text data
    final_prompt = prompt.format(query=query, document=doc)

    # Generate the completion by interacting with the language model API
    completion = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {
                "role": "user",
                "content": final_prompt
            }
        ],
        temperature=0,  # Control the randomness of the output (lower means less random)
        max_tokens=1024,  # Limit the response length
        top_p=1,  # Nucleus sampling parameter (1 means only the most likely tokens are considered)
        stream=True,  # Enable streaming of the response chunks
        stop=None
    )

    # Initialize an empty string to accumulate the response content
    answer = ''''''
    for chunk in completion:
        # Append each chunk of content to the answer string
        answer += chunk.choices[0].delta.content or ""
    
    # Return the dictionary containing the generated questions
    return answer

generate_questions("How is the sun formed?", "John went to a park the other day for a stroll", client)

KeyboardInterrupt: 

In [24]:
from sentence_transformers import CrossEncoder


cross_encoder = CrossEncoder(
        "cross-encoder/ms-marco-TinyBERT-L-2-v2", max_length=512, device="cpu"
    )
    # Format the timeline header and article for cross encoder
unranked_articles = [("I love korea", "I love jeju, korea")]
    
# Predict similarity scores
similarity_scores = cross_encoder.predict(unranked_articles).tolist()
# return similarity_scores
similarity_scores

[9.175457000732422]